In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_test.tsv
!wget https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_train.tsv
!pip install git+https://github.com/csebuetnlp/normalizer
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install torch
!pip install scikit-learn
# !pip install --upgrade accelerate
import logging
import os
import random
import sys
from normalizer import normalize
from dataclasses import dataclass, field
from typing import Optional
import pandas as pd
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
import torch
from sklearn.model_selection import StratifiedKFold

import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from transformers.utils.logging import set_verbosity_info
from transformers.modeling_outputs import SequenceClassifierOutput


logger = logging.getLogger(__name__)

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)
train_file = 'blp25_hatespeech_subtask_1C_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1C_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1C_test.tsv'
import os
os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    output_dir="./banglabert/",
    overwrite_output_dir=True,
    remove_unused_columns=True,
    local_rank= 1,
    load_best_model_at_end=True,
    save_total_limit=2,
    save_strategy="no",
    report_to=None,
    warmup_ratio=0.1,
    weight_decay=0.01,
    fp16=False
)

max_train_samples = None
max_eval_samples=None
max_predict_samples=None
max_seq_length = 512
batch_size = 16
set_verbosity_info()

log_level = training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f" distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")
model_name = 'csebuetnlp/banglabert'
set_seed(training_args.seed)
hate_type_map = {'None': 0, 'Religious Hate': 1, 'Sexism': 2, 'Political Hate': 3, 'Profane': 4, 'Abusive': 5}
severity_map = {'Little to None': 0, 'Mild': 1, 'Severe': 2}
to_whom_map = {'None': 0, 'Individual': 1, 'Organization': 2, 'Community': 3, 'Society': 4}
id2hate = {v: k for k, v in hate_type_map.items()}
id2sev = {v: k for k, v in severity_map.items()}
id2to = {v: k for k, v in to_whom_map.items()}

# Load training and validation data
train_df = pd.read_csv(train_file, sep='\t')

train_df['hate_type'] = train_df['hate_type'].fillna('None')
train_df['to_whom'] = train_df['to_whom'].fillna('None')
train_df['hate_type'] = train_df['hate_type'].map(hate_type_map).astype(int)
train_df['hate_severity'] = train_df['hate_severity'].map(severity_map).astype(int)
train_df['to_whom'] = train_df['to_whom'].map(to_whom_map).astype(int)

validation_df = pd.read_csv(validation_file, sep='\t')
validation_df['hate_type'] = validation_df['hate_type'].replace('nan', 'None').fillna('None')
validation_df['to_whom'] = validation_df['to_whom'].replace('nan', 'None').fillna('None')
validation_df['hate_type'] = validation_df['hate_type'].map(hate_type_map).astype(int)
validation_df['hate_severity'] = validation_df['hate_severity'].map(severity_map).astype(int)
validation_df['to_whom'] = validation_df['to_whom'].map(to_whom_map).astype(int)

# Combine training and validation data for cross-validation
combined_df = pd.concat([train_df, validation_df], ignore_index=True)
combined_dataset = Dataset.from_pandas(combined_df)

# Load test data separately
test_df = pd.read_csv(test_file, sep='\t')
test_dataset = Dataset.from_pandas(test_df)

# Create initial dataset dict for preprocessing
raw_datasets = DatasetDict({
    "combined": combined_dataset,
    "test": test_dataset
})

for key in raw_datasets.keys():
    logger.info(f"loading a local file for {key}")
    
print(f"Combined dataset size: {len(combined_dataset)}")
print(f"Test dataset size: {len(test_df)}")
len(test_df['id'])
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

class MultiTaskModel(torch.nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        self.config = AutoConfig.from_pretrained(model_name)
        hidden_size = self.config.hidden_size
        self.hate_type_head = torch.nn.Linear(hidden_size, len(hate_type_map))
        self.severity_head = torch.nn.Linear(hidden_size, len(severity_map))
        self.to_whom_head = torch.nn.Linear(hidden_size, len(to_whom_map))

    def forward(self, input_ids, attention_mask=None, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        hate_type_logits = self.hate_type_head(pooled_output)
        severity_logits = self.severity_head(pooled_output)
        to_whom_logits = self.to_whom_head(pooled_output)
        loss = None
        if labels is not None:
            hate_type_labels = labels[:, 0]
            severity_labels = labels[:, 1]
            to_whom_labels = labels[:, 2]
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(hate_type_logits, hate_type_labels.long()) + \
                   loss_fct(severity_logits, severity_labels.long()) + \
                   loss_fct(to_whom_logits, to_whom_labels.long())
        return SequenceClassifierOutput(
            loss=loss,
            logits=(hate_type_logits, severity_logits, to_whom_logits),
        )

non_label_column_names = [name for name in raw_datasets["combined"].column_names if name != "labels"]
sentence1_key= 'text'

# Padding strategy
padding = "max_length"

if max_seq_length > tokenizer.model_max_length:
    logger.warning(
        f"The max_seq_length passed ({max_seq_length}) is larger than the maximum length for the"
        f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
    )
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

def preprocess_function(examples):
    # Normalize the texts
    normalized_texts = [
        normalize(
            text,
            unicode_norm="NFKC",
            punct_replacement=None,
            url_replacement=None,
            emoji_replacement=None,
            apply_unicode_norm_last=True
        ) for text in examples[sentence1_key]
    ]
    # Tokenize the texts
    args = (normalized_texts,)
    result = tokenizer(*args, padding=padding, max_length=max_seq_length, truncation=True)
    if "hate_type" in examples:
        result["labels"] = [[l1, l2, l3] for l1, l2, l3 in zip(examples["hate_type"], examples["hate_severity"], examples["to_whom"])]
    return result
# Preprocess the datasets
raw_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)
# Prepare combined dataset for cross-validation
combined_dataset = raw_datasets["combined"]
predict_dataset = raw_datasets["test"]

# Extract features and labels for StratifiedKFold
# For stratification, use hate_type as primary
X = np.arange(len(combined_dataset))
y = np.array(combined_dataset["hate_type"])

print(f"Total samples for cross-validation: {len(y)}")
print(f"Label distribution: {np.bincount(y)}")

# Initialize StratifiedKFold
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=training_args.seed)

# Store results for each fold
fold_results = []
fold_probs = []

# Initialize trainer components
accuracy = evaluate.load("accuracy")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def compute_metrics(p: EvalPrediction):
    hate_preds = np.argmax(p.predictions[0], axis=1)
    sev_preds = np.argmax(p.predictions[1], axis=1)
    to_preds = np.argmax(p.predictions[2], axis=1)
    hate_labels = p.label_ids[:,0]
    sev_labels = p.label_ids[:,1]
    to_labels = p.label_ids[:,2]
    hate_acc = accuracy.compute(predictions=hate_preds, references=hate_labels)['accuracy']
    sev_acc = accuracy.compute(predictions=sev_preds, references=sev_labels)['accuracy']
    to_acc = accuracy.compute(predictions=to_preds, references=to_labels)['accuracy']
    return {'hate_accuracy': hate_acc, 'severity_accuracy': sev_acc, 'to_whom_accuracy': to_acc}

# Cross-validation loop
for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"\n{'='*50}")
    print(f"FOLD {fold + 1}/{n_splits}")
    print(f"{'='*50}")
    
    # Create train and validation datasets for this fold
    train_dataset = combined_dataset.select(train_idx.tolist())
    eval_dataset = combined_dataset.select(val_idx.tolist())
    
    # Remove ID columns if present
    train_dataset = train_dataset.remove_columns("id") if "id" in train_dataset.column_names else train_dataset
    eval_dataset = eval_dataset.remove_columns("id") if "id" in eval_dataset.column_names else eval_dataset
    
    print(f"Train size: {len(train_dataset)}, Validation size: {len(eval_dataset)}")
    
    # Initialize model for this fold
    model = MultiTaskModel(model_name)
    
    # Update training arguments for this fold
    fold_training_args = training_args
    fold_training_args.output_dir = f"{training_args.output_dir}fold_{fold+1}/"
    fold_training_args.seed = training_args.seed + fold  # Different seed for each fold
    
    # Initialize trainer for this fold
    trainer = Trainer(
        model=model,
        args=fold_training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    
    # Train the model
    print(f"Training fold {fold + 1}...")
    train_result = trainer.train()
    
    # Evaluate the model
    print(f"Evaluating fold {fold + 1}...")
    eval_result = trainer.evaluate()
    
    # Store results
    fold_results.append({
        'fold': fold + 1,
        'train_loss': train_result.training_loss if hasattr(train_result, 'training_loss') else train_result.metrics.get('train_loss', None),
        'eval_loss': eval_result['eval_loss'],
        'eval_hate_accuracy': eval_result['eval_hate_accuracy'],
        'eval_severity_accuracy': eval_result['eval_severity_accuracy'],
        'eval_to_whom_accuracy': eval_result['eval_to_whom_accuracy']
    })
    
    # Generate predictions on test set for this fold
    print(f"Predicting with fold {fold + 1} model...")
    test_predictions = trainer.predict(predict_dataset.remove_columns("id") if "id" in predict_dataset.column_names else predict_dataset)
    probs = [torch.softmax(torch.tensor(logits), dim=-1).numpy() for logits in test_predictions.predictions]
    fold_probs.append(probs)
    
    # Clean up to save memory
    del model, trainer
    torch.cuda.empty_cache() if torch.cuda.is_available() else None
    
    print(f"Fold {fold + 1} - Hate Accuracy: {eval_result['eval_hate_accuracy']:.4f}, Severity Accuracy: {eval_result['eval_severity_accuracy']:.4f}, To Whom Accuracy: {eval_result['eval_to_whom_accuracy']:.4f}")
    
print(f"\n{'='*50}")
print("CROSS-VALIDATION COMPLETED")
print(f"{'='*50}")

# Analyze cross-validation results
results_df = pd.DataFrame(fold_results)
print("\nCross-Validation Results:")
print(results_df)

# Calculate average performance metrics
avg_train_loss = results_df['train_loss'].mean() if 'train_loss' in results_df else 0
avg_eval_loss = results_df['eval_loss'].mean()
avg_hate_acc = results_df['eval_hate_accuracy'].mean()
avg_sev_acc = results_df['eval_severity_accuracy'].mean()
avg_to_acc = results_df['eval_to_whom_accuracy'].mean()
std_hate_acc = results_df['eval_hate_accuracy'].std()
std_sev_acc = results_df['eval_severity_accuracy'].std()
std_to_acc = results_df['eval_to_whom_accuracy'].std()

print(f"\nAverage Results Across {n_splits} Folds:")
print(f"Average Training Loss: {avg_train_loss:.4f}")
print(f"Average Validation Loss: {avg_eval_loss:.4f}")
print(f"Average Hate Type Accuracy: {avg_hate_acc:.4f} ± {std_hate_acc:.4f}")
print(f"Average Severity Accuracy: {avg_sev_acc:.4f} ± {std_sev_acc:.4f}")
print(f"Average To Whom Accuracy: {avg_to_acc:.4f} ± {std_to_acc:.4f}")

# Ensemble predictions
hate_probs_folds = np.array([probs[0] for probs in fold_probs])
sev_probs_folds = np.array([probs[1] for probs in fold_probs])
to_probs_folds = np.array([probs[2] for probs in fold_probs])
ensemble_probs = (np.mean(hate_probs_folds, axis=0),
                  np.mean(sev_probs_folds, axis=0),
                  np.mean(to_probs_folds, axis=0))

# Final ensemble prediction
hate_probs, sev_probs, to_probs = ensemble_probs
final_hate_preds = np.argmax(hate_probs, axis=1)
final_sev_preds = np.argmax(sev_probs, axis=1)
final_to_preds = np.argmax(to_probs, axis=1)

# Generate predictions
import os
os.makedirs(training_args.output_dir, exist_ok=True)
logger.info("*** Predict ***")
ids = test_df['id']
output_predict_file = os.path.join(training_args.output_dir, f"subtask_1C.tsv")

# Write predictions in the required format
with open(output_predict_file, "w") as writer:
    logger.info(f"***** Predict results *****")
    writer.write("id\thate_type\thate_severity\tto_whom\tmodel\n")
    for index in range(len(final_hate_preds)):
        h = id2hate[final_hate_preds[index]]
        s = id2sev[final_sev_preds[index]]
        t = id2to[final_to_preds[index]]
        writer.write(f"{ids[index]}\t{h}\t{s}\t{t}\t{model_name}\n")

print(f"\nPredictions saved to '{output_predict_file}'")

--2025-09-15 15:40:39--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 634005 (619K) [text/plain]
Saving to: ‘blp25_hatespeech_subtask_1C_dev.tsv’

blp25_hatespeech_su 100%[===================>] 619.15K  --.-KB/s    in 0.04s   

2025-09-15 15:40:40 (14.2 MB/s) - ‘blp25_hatespeech_subtask_1C_dev.tsv’ saved [634005/634005]

--2025-09-15 15:40:40--  https://raw.githubusercontent.com/AridHasan/blp25_task1/refs/heads/main/data/subtask_1C/blp25_hatespeech_subtask_1C_test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.g

2025-09-15 15:42:46.549906: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757950966.882114      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757950966.972403      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Combined dataset size: 38034
Test dataset size: 10200


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

...

Step,Training Loss
500,2.776600
1000,2.172700
1500,1.953700


[INFO|trainer.py:2676] 2025-09-15 18:18:34,407 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


[INFO|trainer.py:934] 2025-09-15 18:18:34,445 >> The following columns in the Evaluation set don't have a corresponding argument in `MultiTaskModel.forward` and have been ignored: to_whom, hate_type, token_type_ids, hate_severity, text. If to_whom, hate_type, token_type_ids, hate_severity, text are not expected by `MultiTaskModel.forward`,  you can safely ignore this message.
[INFO|trainer.py:4327] 2025-09-15 18:18:34,451 >> 
***** Running Evaluation *****
[INFO|trainer.py:4329] 2025-09-15 18:18:34,451 >>   Num examples = 12678
[INFO|trainer.py:4332] 2025-09-15 18:18:34,452 >>   Batch size = 32


Evaluating fold 3...


[INFO|trainer.py:934] 2025-09-15 18:23:14,256 >> The following columns in the test set don't have a corresponding argument in `MultiTaskModel.forward` and have been ignored: token_type_ids, text. If token_type_ids, text are not expected by `MultiTaskModel.forward`,  you can safely ignore this message.
[INFO|trainer.py:4327] 2025-09-15 18:23:14,259 >> 
***** Running Prediction *****
[INFO|trainer.py:4329] 2025-09-15 18:23:14,260 >>   Num examples = 10200
[INFO|trainer.py:4332] 2025-09-15 18:23:14,260 >>   Batch size = 32


Predicting with fold 3 model...
Fold 3 - Hate Accuracy: 0.7196, Severity Accuracy: 0.7469, To Whom Accuracy: 0.7241

CROSS-VALIDATION COMPLETED

Cross-Validation Results:
   fold  train_loss  eval_loss  eval_hate_accuracy  eval_severity_accuracy  \
0     1    2.290406   2.100453            0.718646                0.738918   
1     2    2.256069   2.092196            0.718489                0.742783   
2     3    2.280356   2.061700            0.719593                0.746884   

   eval_to_whom_accuracy  
0               0.717069  
1               0.723142  
2               0.724089  

Average Results Across 3 Folds:
Average Training Loss: 2.2756
Average Validation Loss: 2.0848
Average Hate Type Accuracy: 0.7189 ± 0.0006
Average Severity Accuracy: 0.7429 ± 0.0040
Average To Whom Accuracy: 0.7214 ± 0.0038

Predictions saved to './banglabert/fold_1/fold_2/fold_3/subtask_1C.tsv'
